In [7]:
import os
import torch
from torch.utils.data import random_split
import torchio as tio
from pathlib import Path
import importlib
from utils import validate_segmentation, plot_prediction, dice_score


import torchio_utils
importlib.reload(torchio_utils)
from torchio_utils import torchio_compose_train, plot_example


#check if colab is being used
if os.path.exists('/content'):
    print('Running on CoLab')
    from google.colab import drive
    drive.mount('/content/drive')
    torch.cuda.empty_cache()

dataset_path = 'data.zip'
dataset_dir_name = 'data'
dataset_dir = Path(dataset_dir_name)

if not dataset_dir.is_dir():
    !curl --silent --output {dataset_path} --location {dataset_url} 
    !unzip -qq {dataset_path}
!tree -d {dataset_dir_name}

data
├── controls
│   └── imgs
└── patients
    ├── imgs
    └── labels

5 directories


In [8]:
images_dir = dataset_dir / 'patients/imgs'
labels_dir = dataset_dir / 'patients/labels'
controls_dir = dataset_dir / 'controls/imgs'
image_paths = sorted(images_dir.glob('*.png'))
label_paths = sorted(labels_dir.glob('*.png'))
control_paths = sorted(controls_dir.glob('*.png'))
original_size = 182
control_size = 198
dataset_ID = 3

print(len(image_paths), len(control_paths))

182 198


In [9]:

torchio_compose_train(image_paths, label_paths, control_paths, 
                                cropsize=(400,991),
                                save_inverted_imgs=False,
                                train_size = 182+198,
                                include_controls=True,
                                dataset_ID=dataset_ID)

Found 182 subjects
Saving 182 subjects to nnUNet_raw/Dataset003...


100%|██████████| 182/182 [00:22<00:00,  8.09it/s]


Augmenting dataset with 220 subjects


100%|██████████| 220/220 [02:22<00:00,  1.54it/s]


Adding 198 control subjects
Saving 198 controls to nnUNet_raw/Dataset003...


100%|██████████| 198/198 [00:25<00:00,  7.70it/s]


True